**Cargar modelo de embeddings para convertir los dato númericos restantes a datos semánticos**

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-base-en-v1.5",
    model_kwargs={"device": "cpu", "trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},
)

**Cargar colección de datos procesados con el main.py**   
Leemos el archivo de datos procesados mediante un dump a un archivo .bson,   
puesto que queremos realizar un menor número de peticiones a la base de datos.

In [ ]:
import bson

with open("embedded.bson", "rb") as f:
    data = bson.decode_all(f.read())

print(len(data))

Para el caso de la CNN calcularemos los parámetros para poder estandarizarlos más adelante en tiempo de ejecución.

In [ ]:
import numpy as np
expYears = []
expYearsManagement = []
avgTimeInJob = []
for i in range(len(data)):
    expYears.append(data[i]["expYears"])
    expYearsManagement.append(data[i]["expYearsManagement"])
    avgTimeInJob.append(data[i]["avgTimeInJob"])
expYears = np.array(expYears).reshape(-1, 1)
expYearsManagement = np.array(expYearsManagement).reshape(-1, 1)
avgTimeInJob = np.array(avgTimeInJob).reshape(-1, 1)

# Compute the minimum and maximum of the data
min_val = np.min(expYears)
max_val = np.max(expYears)

min_val_management = np.min(expYearsManagement)
max_val_management = np.max(expYearsManagement)

min_val_avgTimeInJob = np.min(avgTimeInJob)
max_val_avgTimeInJob = np.max(avgTimeInJob)

# Define a function to scale data
def min_max_scale(data, min_val, max_val):
    return (data - min_val) / (max_val - min_val)


**Preparar una función para convertir los datos númericos restantes a datos semánticos**     
Originalmente recibimos algunos datos que son escalares númericos, como los años de experiencia, estos serán convertidos a datos semánticos. Encontramos que la mejor forma de conservar la numeralidad en la forma de embeddings era primero pasar los años a meses y luego a una cadena de texto.  
Esto lo hacemos con una librería de python que dado un entero, lo convierte a su forma de texto en inglés.

In [ ]:
from num2words import num2words
def monthsofexperience(expYears):
    months = int(expYears * 12)
    return f"The candidate has {num2words(months)} months of labor experience"

# monthsofexperience(2) -> 'The candidate has twenty-four months of labor experience'

En el anterior bloque de código, se puede ver la primera función de conversión de datos númericos a datos semánticos.   
Como ejemplo si pasamos como argumento 2, la función devolverá un string con el texto *"The candidate has twenty-four months of labor experience"*.   
Definimos el resto de funciones que funcionan bajo el mismo principio, pero para distintos campos númericos.

In [ ]:
def expYearsEmbedding(expYears):
    if expYears >= 5:
        temp =monthsofexperience(expYears) + ", so he has enough experience to be a manager"
    else:
        temp = monthsofexperience(expYears) + ", so he is not qualified to be a manager"
    return np.array(embeddings.embed_documents([temp])[0])
    
def expYearsEmbedding2(expYearsManagement):
    if expYearsManagement >= 5:
        temp = monthsofexperience(expYearsManagement) + ", so he has enough experience to have an executive position"
    else:
        temp = monthsofexperience(expYearsManagement) + ", so he is not qualified to have an executive position"
    return np.array(embeddings.embed_documents([temp])[0])

def avgTimeInJobEmb(avgTimeInJob):
    temp = f"The candidate has an average of {num2words(avgTimeInJob)} months in each job"
    return np.array(embeddings.embed_documents([temp])[0])

def management_position(management_position):
    if management_position:
        temp = "One of the recent jobs of the candidate was in a management position"
    elif management_position == False:
        temp = "One of the recent jobs of the candidate did not involve management activities"
    else:
        temp = "The candidate didn't provide information about his recent jobs"
        
    return np.array(embeddings.embed_documents([temp])[0])

def education_level(education_level):
    if education_level < 0:
        temp = "The candidate didn't provide information about his education level"
    elif education_level == 0:
        temp = "The candidate has a high school education level"
    elif education_level == 1:
        temp = "The candidate has a college education level"
    elif education_level == 2:
        temp = "The candidate has a postgraduate education level"
    else:
        temp = "The candidate has a doctorate education level"
    return np.array(embeddings.embed_documents([temp])[0])

**Definir constantes para datos faltantes**    
Hemos decidido utilizar siempre los últimos 3 trabajos de un candidato, sin embargo, si un candidato tiene menos de 3 trabajos, completaremos con las definidas a continuación.    
Lo mismo aplicará a la educación del candidato, siempre trataremos de obtener información acerca de su grado universitario, y sobre su educación más alta por encima del grado universitario. Si no se tiene información, también completaremos con las definidas a continuación.

In [ ]:
NA_JOB_CONST = np.array(embeddings.embed_documents(["No more jobs where found for this candidare"])[0])
NA_CONST = np.array(embeddings.embed_documents(["Not available information"])[0])

**Convertir la información sobre la carpeta al momento de procesar los datos en la etiqueta para aprendizaje supervisado**    
Al momento de procesar los CVs, se guardó la ruta de la carpeta donde se encontraba el CV, esta información será utilizada para etiquetar los datos.

In [ ]:
def labeler(label):
    if "Especialista" in label:
        return 0
    elif "Gerente" in label:
        return 1
    elif "Director" in label:
        return 2

**Crear una función para obtener las matrices qeu alimentan a los modelos**    
Con las funciones previamente definidas, podemos obtener las matrices que alimentarán a los modelos.    
Hemos definido 15 características a utilizar para candidato que consideramos relevantes para su clasificación en una de las 3 categorías. Estas características han sido cuidadosamente seleccionadas para evitar tener sesgos en el modelo. Y siempre estarán en un lenguaje neutro y omiten deliberadamente información de carácter personal, puesto que han sido reescritas de forma que se omiten detalles particulares.
Las características son las siguientes:
1. Años de experiencia laboral
2. Años en puestos de liderazgo
3. Tiempo promedio en cada trabajo
4. Información indicando el nivel de educación más alto (High School, College, Postgraduate, NA)
5. Titulo de educación más alta (Por encima de grado universitario, no se incluye el nombre de la universidad)
6. Titulo de grado universitario (No se incluye el nombre de la universidad)
7. Titulo de trabajo último trabajo (No se incluye el nombre de la empresa de forma deliberada)
8. Resumen de las responsabilidades del último trabajo
9. Categorización sobre si el último trabajo involucra liderazgo
10. Titulo del penúltimo trabajo (No se incluye el nombre de la empresa de forma deliberada)
11. Resumen de las responsabilidades del penúltimo trabajo
12. Categorización sobre si el penúltimo trabajo involucra liderazgo
13. Titulo del antepenúltimo trabajo (No se incluye el nombre de la empresa de forma deliberada)
14. Resumen de las responsabilidades del antepenúltimo trabajo
15. Categorización sobre si el antepenúltimo trabajo involucra liderazgo


In [ ]:
def to_matrix(bson):
    totalWorks = len(bson["work"])
    work_title = []
    work_brief = []
    work_management = []
    for i in range(1,4):
        if totalWorks-i < 0:
            work_title.append(NA_JOB_CONST)
            work_brief.append(NA_JOB_CONST)
            work_management.append(NA_JOB_CONST)
        else:
            work_title.append(bson["work"][totalWorks-i]["title"])
            work_brief.append(bson["work"][totalWorks-i]["brief"])
            if bson["work"][totalWorks-i]["management"] == 0:
                work_management.append(management_position(False))
            else:
                work_management.append(management_position(True))
    if bson["bachelor"] != None:
        bachelor_title = bson["bachelor"]["title"]
    else:
        bachelor_title = NA_CONST

    if bson["maxEducation"] != None:
        maxEducation_title = bson["maxEducation"]["title"]
    else:
        maxEducation_title = NA_CONST
        
    expYears = expYearsEmbedding(bson["expYears"])
    #expYearsNumeric = np.ones(expYears.shape) * min_max_scale(bson["expYears"], min_val, max_val)
    expYearsManagement = expYearsEmbedding2(bson["expYearsManagement"])
    #expYearsManagementNumeric = np.ones(expYearsManagement.shape) * min_max_scale(bson["expYearsManagement"], min_val_management, max_val_management)
    avgTimeInJob = avgTimeInJobEmb(bson["avgTimeInJob"])
    #avgTimeInJobNumeric = np.ones(avgTimeInJob.shape) * min_max_scale(bson["avgTimeInJob"], min_val_avgTimeInJob, max_val_avgTimeInJob)
    highestEducation = education_level(bson["highestEducation"])

    # Put all the data together into a giant np.array where each variable is a column
    data = np.vstack([expYears,
                      #expYearsNumeric, 
                    expYearsManagement, 
                    #expYearsManagementNumeric,
                    avgTimeInJob, 
                    #avgTimeInJobNumeric,
                    highestEducation,
                    bachelor_title, 
                    maxEducation_title, 
                    work_title[0], 
                    work_brief[0],
                    work_management[0], 
                    work_title[1],
                    work_brief[1],
                    work_management[1], 
                    work_title[2], 
                    work_brief[2],
                    work_management[2]])
    return (data, labeler(bson["label"]), bson["file"])

Lo último que queda es ejecutar la función dentro de un bucle que recorra todos los datos procesados y guardarlos en un archivo .npy que tendrá las siguientes dimensiones:    
(Número de datos, 15, 768)

In [ ]:
from random import randint
# Añadimos un randint para evitar colisiones en los nombres de los archivos
# En este caso guardamos los archivos en la carpeta Data_matrix por si se necesitan tener por separado
def flujo(bson):
    try:
        a = to_matrix(bson)
        filename = a[2].split("/")[0]
        with open(f"Data_matrix/{a[1]}/{filename}{randint(1,100)}.csv", "wb") as f:
            np.savetxt(f, a[0], delimiter=",")
    except:
        print("Error with ", bson["file"])

In [ ]:
import os
from joblib import Parallel, delayed
Parallel(n_jobs=-1, prefer="threads")(delayed(flujo)(bson) for bson in data)

def load_data():
    X = []
    y = []
    for label in os.listdir("Data_matrix"):
        for file in os.listdir(f"Data_matrix/{label}"):
            data = np.loadtxt(f"Data_matrix/{label}/{file}", delimiter=",")
            X.append(data)
            y.append(int(label))
    return np.array(X), np.array(y)

X, Y = load_data()

X = np.load("X.npy")
Y = np.load("Y.npy")